In [23]:
import Models , LoadBatches
from keras.models import load_model
import glob
import cv2
import numpy as np
import random

In [24]:
n_classes = 2
model_name = "fcn32"
images_path = "data/images_test/"
input_width =  224
input_height = 224
epoch_number = 4

In [25]:
modelFns = { 'fcn32':Models.FCN32.FCN32   }
modelFN = modelFns[ model_name ]

In [26]:
m = modelFN( n_classes , input_height=input_height, input_width=input_width   )

m.load_weights(  "weight" + "." + str(  epoch_number )  )

m.compile(loss='categorical_crossentropy',
      optimizer= 'adadelta' ,
      metrics=['accuracy'])

koko (None, 2, 256, 256)


In [27]:
output_height = m.outputHeight
output_width = m.outputWidth
score=0

In [28]:
images = glob.glob( images_path + "*.jpg"  ) + glob.glob( images_path + "*.png"  ) +  glob.glob( images_path + "*.jpeg"  )
images.sort()

colors = [  ( random.randint(0,255),random.randint(0,255),random.randint(0,255)   ) for _ in range(n_classes)  ]

In [29]:
# running the trained model on 500 unseen images
for imgName in images:
	outName = imgName.replace( images_path ,  "data/predictions/" )
	X = LoadBatches.getImageArr(imgName , input_width  , input_height  )
	pr = m.predict( np.array([X]) )[0]
	score += m.evaluate(np.array([X]),pr)[1] 
	pr = pr.reshape(( output_height ,  output_width , n_classes ) ).argmax( axis=2 )
	seg_img = np.zeros( ( output_height , output_width , 3  ) )
	for c in range(n_classes):
		seg_img[:,:,0] += ( (pr[:,: ] == c )*( colors[c][0] )).astype('uint8')
		seg_img[:,:,1] += ((pr[:,: ] == c )*( colors[c][1] )).astype('uint8')
		seg_img[:,:,2] += ((pr[:,: ] == c )*( colors[c][2] )).astype('uint8')
	seg_img = cv2.resize(seg_img  , (input_width , input_height ))
	cv2.imwrite(  outName , seg_img )

In [30]:
# the accuracy is at pixel level
print ("Accuracy: " + score/500)

Accuracy: 0.72913333
